# Introduction to Azure OpenAI Function Calling


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling


## Time

You should expect to spend 2 minutes running this sample.


## About this example

The objective of the provided Python file is to learn about Azure OpenAI Function Calling using the Azure OpenAI API.


## Installation

Refer to the README.md file in this folder for installation instructions.


## Parameters


In [1]:
import os
import json

from typing import Any, Callable, Dict

import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML

from sales_data import SalesData

load_dotenv(".env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

system_message = None

In [ ]:
sales_data = SalesData()
sales_data.connect()
db_info = sales_data.get_database_info()
print(db_info)

In [3]:
def log_message(message: str):
    display(
        HTML(f'<span style="color: red;"><strong>{message}</strong></span>'))

In [4]:
def print_in_color(key, value):
    display(HTML(f"<span style='color: green;font-weight: bold;font-size: medium;'>{key}</span> "
            f"<span style='color: blue;font-weight: bold;font-size: medium;'>{value}</span>"))

In [5]:
def display_wrapped(message: str):
    wrapped_content = f'<pre style="white-space: pre-wrap; word-wrap: break-word;">{message}</pre>'
    display(HTML(wrapped_content))

### Create an AzureOpenAI client


In [6]:
client = AzureOpenAI(api_key=api_key, api_version=api_version,
                     azure_endpoint=api_endpoint)

### Define the Assistant tools

The tools list defines the tools that the OpenAI Chat Completion will look for when generating a response. It's then up to the application to decide how to use the response. In this example corresponding Python functions are defined for each tool and the response is passed to the appropriate function.

In [7]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "fetch_sales_data_using_sqlite_query",
            "description": "This function is used to answer user questions about Contoso sales data by executing SQLite queries against the database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                            The input should be a well-formed SQLite query to extract information based on the user's question.
                            The query result will be returned as plain text, not in JSON format.
                        """,
                    }
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        },
    },
]

In [8]:
function_map: Dict[str, Callable[[Any], pd.DataFrame]] = {
    "fetch_sales_data_using_sqlite_query": lambda args: sales_data.fetch_sales_data_using_sqlite_query_pandas(args["query"]),
}

### Process Function calling

Responsible for calling the appropriate function based on the tool detected in the response. The function is called with the response and the tool as arguments.


In [9]:
def call_functions(tool_calls) -> None:

    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        print_in_color("Executing function tool call", "")
        print_in_color("Function Name:", func_name)
        print_in_color("Arguments:", arguments)

        function = function_map.get(func_name)
        if not function:
            raise ValueError(f"Unknown function: {func_name}")

        result_df = function(arguments)
        display(result_df)

## Instructions for role system assignment

Defines the system message for the role system assignment.



In [ ]:
system_message = (
    "You are a sales analysis assistant for Contoso. "
    "Please be polite, professional, helpful, and friendly. "
    "Use the `fetch_sales_data_using_sqlite_query` function to execute sales data queries, defaulting to aggregated data unless a detailed breakdown is requested. The function returns JSON-formatted results. "
    f"Refer to the Contoso sales database schema: {db_info}. ",
    "If a question is not related to Contoso sales data or you cannot answer the question, "
    "then simply say, 'I can't answer that question. Please contact IT for more assistance.' "
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer."
)
print(system_message)

## Construct the Assistant and send to Azure OpenAI


In [11]:
def process_message(question: str):

    messages = [{"role": "system", "content": str(system_message)}, {
        "role": "user", "content": question}]

    try:
        response = client.chat.completions.create(
            model=api_deployment_name,
            messages=messages,
            tools=tools_list,
            temperature=0.2,
            max_tokens=512,
        )

        response_message = response.choices[0].message
        tool_calls = getattr(response_message, "tool_calls", [])

        if tool_calls:
            call_functions(tool_calls)
        else:
            display_wrapped(response_message.content)

    except Exception as e:
        log_message(f"An error occurred: {e}")

In [ ]:
process_message("What is the revenue for the region of Africa?")

In [ ]:
process_message(
    "What is the revenue for each of the americas for 2023? Be sure to include the year.")

In [ ]:
process_message("What is the revenue for all regions?")

In [ ]:
process_message("What is the revenue for all regions? Include the year.")

In [ ]:
process_message("What were the sales for April 2023?")

In [ ]:
process_message("What is the meaning of life?")

In [ ]:
sales_data.close()